In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
from network_models import train_neural_network, inference_step

Using TensorFlow backend.


In [3]:
from network_models import TXYZnet

In [4]:
model = TXYZnet(num_classes=2)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
txyz_input (InputLayer)      (None, 75, 1, 16, 15, 18) 0         
_________________________________________________________________
td_txyz_block1_conv1 (TimeDi (None, 75, 32, 16, 15, 18 55328     
_________________________________________________________________
td_txyz_block1_conv2 (TimeDi (None, 75, 32, 16, 15, 18 1769504   
_________________________________________________________________
td_txyz_block1_pool (TimeDis (None, 75, 32, 8, 8, 9)   0         
_________________________________________________________________
xy_squashing (Lambda)        (None, 75, 32, 9)         0         
_________________________________________________________________
permute_1 (Permute)          (None, 32, 75, 9)         0         
_________________________________________________________________
tz_block3_conv1 (Conv2D)     (None, 64, 75, 9)         294976    
__________

In [6]:
from multi_files_generator import data_generator, metadata_generator

In [7]:
from get_iterations import get_n_iterations, get_class_weights

In [8]:
train_test_dir = os.path.join("train_test_files","cosz")
fnames_train =[os.path.join(train_test_dir, "Xy_train{}_sel5_doms.npz".format(i+1)) for i in range(100)]
fnames_test =[os.path.join(train_test_dir, "Xy_test{}_sel5_doms.npz".format(i+1)) for i in range(100)]
fnames_val =[os.path.join(train_test_dir, "Xy_val{}_sel5_doms.npz".format(i+1)) for i in range(100)]

In [9]:
n_files=100
batch_size = 32
steps_per_epoch, n_events = get_n_iterations(fnames_train[:n_files], batch_size=batch_size)
print(steps_per_epoch, n_events)
validation_steps, n_evts_val = get_n_iterations(fnames_val[:n_files], batch_size=batch_size)
print(validation_steps, n_evts_val)
prediction_steps, n_evts_test = get_n_iterations(fnames_test[:n_files], batch_size=batch_size)
print(prediction_steps, n_evts_test)

(5176, 165610)
(1296, 41451)
(1620, 51818)


In [10]:
cls_weights = {i: v for i, v in enumerate(get_class_weights(fnames_train[:n_files]))}

In [11]:
from keras.utils import to_categorical
def process_cosz(y):
    y[y>0]=1
    y[y<=0]=0
    return to_categorical(y)

def add_channel_dim(X):
    return X[:,:, np.newaxis, ...]
training_generator = data_generator(fnames_train[:n_files], batch_size=batch_size, 
                                    fdata=add_channel_dim, ftarget=process_cosz)

In [12]:
validation_generator = data_generator(fnames_val[:n_files], batch_size=batch_size,
                                     fdata=add_channel_dim, ftarget=process_cosz)

In [ ]:
train_neural_network(model, training_generator, steps_per_epoch, validation_generator, validation_steps,
                     batch_size=batch_size, class_weights=cls_weights, log_suffix="updown")

Epoch 1/100
   2/5176 [..............................] - ETA: 459423s - loss: 4.8291 - acc: 0.4844